In [2]:
import json
import time
import math
import networkx as nx

# =============================
# Helpers (fonctions utilitaires)
# =============================

def digit_sum(node_id: str) -> int:
    # Calcule la somme des chiffres dans l’ID du nœud
    # Exemple : "4.1" -> 4+1 = 5
    return sum(int(ch) for ch in node_id if ch.isdigit())

def depth_of(node_id: str) -> int:
    # Retourne la "profondeur" du nœud = première partie avant le point
    # Exemple : "4.1" -> 4 ; "7" -> 7
    try:
        return int(node_id.split('.')[0])
    except Exception:
        return 0

def immediate_prereqs(graph_adj: dict, failed_nodes: list[str]) -> list[str]:
    """
    Trouve les prérequis immédiats (1 saut) des matières échouées par l'étudiant.
    """
    s = set()
    for fn in failed_nodes:
        s.update(graph_adj.get(fn, []))  # Ajoute les prérequis directs
    return list(sorted(s))  # Tri pour avoir un ordre stable

def build_graph_from_adj(adj: dict) -> nx.DiGraph:
    """
    Construit le graphe orienté prerequisites -> course.
    """
    G = nx.DiGraph()
    for node, prereqs in adj.items():
        for p in prereqs:
            G.add_edge(p, node)
    return G


# =============================
# Métriques de corrélation (comparaison de classements)
# =============================

def spearman(order_a: list[str], order_b: list[str]) -> float:
    """
    Corrélation de Spearman : mesure si les deux classements ont un ordre similaire.
    1.0 = ordres identiques, -1.0 = ordres inverses.
    """
    n = len(order_a)
    if n < 2:
        return 1.0
    rank_a = {x: i for i, x in enumerate(order_a)}
    rank_b = {x: i for i, x in enumerate(order_b)}
    d2_sum = 0
    for x in order_a:
        d = rank_a[x] - rank_b[x]
        d2_sum += d * d
    return 1 - 6 * d2_sum / (n * (n*n - 1))

def kendall_tau(order_a: list[str], order_b: list[str]) -> float:
    """
    Kendall Tau : mesure le nombre de paires concordantes vs discordantes.
    1.0 = mêmes paires ordonnées, -1.0 = ordres complètement opposés.
    """
    n = len(order_a)
    if n < 2:
        return 1.0
    pos_a = {x: i for i, x in enumerate(order_a)}
    pos_b = {x: i for i, x in enumerate(order_b)}
    concordant = 0
    discordant = 0
    total_pairs = n * (n - 1) // 2
    items = order_a
    for i in range(n):
        for j in range(i + 1, n):
            xi, xj = items[i], items[j]
            sa = pos_a[xi] - pos_a[xj]
            sb = pos_b[xi] - pos_b[xj]
            if sa * sb > 0:      # paires dans le même ordre
                concordant += 1
            elif sa * sb < 0:    # paires inversées
                discordant += 1
    return (concordant - discordant) / total_pairs if total_pairs > 0 else 1.0


# =============================
# Autres métriques de distance
# =============================

def footrule_distance_norm(order: list[str], baseline: list[str]) -> float:
    """
    Distance de Spearman Footrule (normalisée 0-1).
    0 = identique au baseline ; 1 = totalement différent.
    """
    n = len(order)
    if n < 2:
        return 0.0
    r = {x: i for i, x in enumerate(order)}
    b = {x: i for i, x in enumerate(baseline)}
    dist = sum(abs(r[x] - b[x]) for x in order)
    max_dist = (n*n) // 2  # distance max
    return dist / max_dist if max_dist > 0 else 0.0

def inversion_rate(order: list[str], baseline: list[str]) -> float:
    """
    Proportion de paires inversées par rapport au baseline.
    0 = même ordre, 1 = complètement inversé.
    """
    n = len(order)
    if n < 2:
        return 0.0
    pos_o = {x: i for i, x in enumerate(order)}
    inv = 0
    total = n * (n - 1) // 2
    items = baseline
    for i in range(n):
        for j in range(i + 1, n):
            xi, xj = items[i], items[j]
            if pos_o[xi] > pos_o[xj]:
                inv += 1
    return inv / total if total > 0 else 0.0

def top_k_overlap(order_a: list[str], order_b: list[str], k: int) -> float:
    """
    Taux de recouvrement entre les k premiers éléments de deux classements.
    1.0 = les mêmes top-k.
    """
    k = min(k, len(order_a), len(order_b))
    if k <= 0:
        return 0.0
    return len(set(order_a[:k]) & set(order_b[:k])) / k


# =============================
# nDCG (Normalized Discounted Cumulative Gain)
# =============================

def make_graded_relevance_from_baseline(baseline: list[str]) -> dict[str, float]:
    """
    Crée un score de pertinence basé sur la position dans le baseline.
    Plus le nœud est haut dans le baseline, plus sa "pertinence" est forte.
    """
    N = len(baseline)
    return {node: float(N - i) for i, node in enumerate(baseline)}

def ndcg_from_relevance(order: list[str], relevance: dict[str, float]) -> float:
    """
    Mesure la qualité d’un classement par rapport à une pertinence de référence.
    1.0 = classement idéal (comme le baseline).
    """
    if not order:
        return 0.0
    dcg = 0.0
    for i, n in enumerate(order):
        rel = relevance.get(n, 0.0)
        gain = (2**rel - 1.0)
        dcg += gain / math.log2(i + 2.0)
    ideal = sorted(order, key=lambda x: -relevance.get(x, 0.0))
    idcg = 0.0
    for i, n in enumerate(ideal):
        rel = relevance.get(n, 0.0)
        gain = (2**rel - 1.0)
        idcg += gain / math.log2(i + 2.0)
    return dcg / idcg if idcg > 0 else 0.0


# =============================
# Algorithmes de classement
# =============================

def baseline_order(nodes: list[str]) -> list[str]:
    """
    Baseline (pas un algorithme d’apprentissage) :
    Tri simple par profondeur ↑, somme des chiffres ↑, puis ID ↑
    """
    return sorted(nodes, key=lambda n: (depth_of(n), digit_sum(n), n))

def pagerank_order(G: nx.DiGraph, nodes: list[str]):
    """
    PageRank : calcule l’importance des nœuds selon le graphe.
    Plus un nœud a de liens entrants importants, plus il est haut.
    """
    start = time.perf_counter()
    pr = nx.pagerank(G, alpha=0.85, max_iter=100, tol=1e-8)
    elapsed_ms = (time.perf_counter() - start) * 1000
    ordered = sorted(nodes, key=lambda n: (-pr.get(n, 0.0), n))
    return ordered, pr, elapsed_ms

def hits_order(G: nx.DiGraph, nodes: list[str]):
    """
    HITS : calcule Autorité et Hub.
    Ici on classe selon l'autorité (importance comme "source d'information").
    """
    start = time.perf_counter()
    try:
        hubs, auth = nx.hits(G, max_iter=500, tol=1e-8, normalized=True)
    except nx.exception.PowerIterationFailedConvergence:
        hubs = {n: 1.0 for n in G.nodes()}
        auth = {n: 1.0 for n in G.nodes()}
    elapsed_ms = (time.perf_counter() - start) * 1000
    ordered = sorted(nodes, key=lambda n: (-auth.get(n, 0.0), n))
    return ordered, hubs, auth, elapsed_ms


# =============================
# Affichage
# =============================

def print_algorithm_block(title: str, ordered_nodes: list[str], metrics: dict, extras: dict | None = None):
    """
    Affiche les résultats d’un algorithme (ordre et métriques).
    """
    print(f"\n— {title} —")
    print("Nodes to repass (ordered):")
    for n in ordered_nodes:
        print(f"  {n}")
    if extras:
        for k, d in extras.items():
            print(f"  {k}:")
            for nk, nv in d.items():
                if isinstance(nv, float):
                    print(f"    {nk}: {nv:.4f}")
                else:
                    print(f"    {nk}: {nv}")
    print("Metrics:")
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"  {k}: {v:.4f}")
        else:
            print(f"  {k}: {v}")


# =============================
# Main (programme principal)
# =============================

def main():
    # Chargement des données
    with open("graph.json", "r", encoding="utf-8") as f:
        graph_adj = json.load(f)
    with open("students.json", "r", encoding="utf-8") as f:
        students = json.load(f)

    # Saisie de l’étudiant
    student_id = input("Entrez l'ID de l'étudiant : ").strip()
    student = next((s for s in students if s["id"] == student_id), None)
    if not student:
        print("Aucun étudiant trouvé avec cet ID.")
        return

    # Construction du graphe
    G = build_graph_from_adj(graph_adj)

    # Nœuds à repasser = prérequis des modules échoués
    failed = student.get("failed_nodes", [])
    candidates = immediate_prereqs(graph_adj, failed)

    print(f"\nNœuds à refaire pour {student['name']} (ID: {student['id']}, Class: {student['class']}):")
    if not candidates:
        print("  (Aucun prérequis direct trouvé.)")
        return

    # ---------- Baseline ----------
    t0 = time.perf_counter()
    base_order = baseline_order(candidates)
    base_time_ms = (time.perf_counter() - t0) * 1000

    # Pertinence graduée dérivée du baseline
    graded_rel = make_graded_relevance_from_baseline(base_order)

    base_metrics = {
        "runtime_ms": base_time_ms,
        "note": "Baseline is a simple deterministic ordering (not a learning algorithm).",
        "ndcg_graded": ndcg_from_relevance(base_order, graded_rel),  # toujours 1.0
        "footrule_norm_vs_self": 0.0,
        "inversion_rate_vs_self": 0.0,
        "top3_overlap_vs_self": 1.0,
        "top5_overlap_vs_self": 1.0
    }
    print_algorithm_block("Baseline (depth ↑, digit_sum ↑)", base_order, base_metrics)

    # ---------- PageRank ----------
    pr_order, pr_scores, pr_time_ms = pagerank_order(G, candidates)
    pr_metrics = {
        "runtime_ms": pr_time_ms,
        "spearman_vs_baseline": spearman(base_order, pr_order),
        "kendall_tau_vs_baseline": kendall_tau(base_order, pr_order),
        "footrule_norm_vs_baseline": footrule_distance_norm(pr_order, base_order),
        "inversion_rate_vs_baseline": inversion_rate(pr_order, base_order),
        "top3_overlap_vs_baseline": top_k_overlap(pr_order, base_order, k=3),
        "top5_overlap_vs_baseline": top_k_overlap(pr_order, base_order, k=5),
        "ndcg_graded_vs_baseline": ndcg_from_relevance(pr_order, graded_rel),
    }
    pr_extras = {
        "PageRank scores (candidates)": {n: pr_scores.get(n, 0.0) for n in pr_order}
    }
    print_algorithm_block("PageRank", pr_order, pr_metrics, extras=pr_extras)

    # ---------- HITS ----------
    hits_ordered, hubs, auth, hits_time_ms = hits_order(G, candidates)
    hits_metrics = {
        "runtime_ms": hits_time_ms,
        "spearman_vs_baseline": spearman(base_order, hits_ordered),
        "kendall_tau_vs_baseline": kendall_tau(base_order, hits_ordered),
        "footrule_norm_vs_baseline": footrule_distance_norm(hits_ordered, base_order),
        "inversion_rate_vs_baseline": inversion_rate(hits_ordered, base_order),
        "top3_overlap_vs_baseline": top_k_overlap(hits_ordered, base_order, k=3),
        "top5_overlap_vs_baseline": top_k_overlap(hits_ordered, base_order, k=5),
        "ndcg_graded_vs_baseline": ndcg_from_relevance(hits_ordered, graded_rel),
    }
    hits_extras = {
        "HITS authority (candidates)": {n: auth.get(n, 0.0) for n in hits_ordered}
    }
    print_algorithm_block("HITS (Authority)", hits_ordered, hits_metrics, extras=hits_extras)

    # ---------- Comparaison globale ----------
    print("\n=== Agreement Between Algorithms (info) ===")
    print(f"Baseline vs PageRank: Spearman={spearman(base_order, pr_order):.3f}  KendallTau={kendall_tau(base_order, pr_order):.3f}")
    print(f"Baseline vs HITS:     Spearman={spearman(base_order, hits_ordered):.3f}  KendallTau={kendall_tau(base_order, hits_ordered):.3f}")
    print(f"PageRank vs HITS:     Spearman={spearman(pr_order, hits_ordered):.3f}    KendallTau={kendall_tau(pr_order, hits_ordered):.3f}")

    # ---------- Recommandation finale ----------
    # Choix du meilleur algorithme (baseline exclue) basé sur nDCG puis Spearman puis vitesse
    algo_scores = {
        "PageRank": (pr_metrics["ndcg_graded_vs_baseline"], pr_metrics["spearman_vs_baseline"], -pr_time_ms),
        "HITS":     (hits_metrics["ndcg_graded_vs_baseline"], hits_metrics["spearman_vs_baseline"], -hits_time_ms),
    }
    best_algo = max(algo_scores.items(), key=lambda kv: kv[1])[0]
    print(f"\n>> Recommendation: {best_algo} (Baseline excluded; ranked by nDCG, then Spearman, then speed)")

if __name__ == "__main__":
    main()


Entrez l'ID de l'étudiant : 003

Nœuds à refaire pour Mohamed Salah (ID: 003, Class: 1):

— Baseline (depth ↑, digit_sum ↑) —
Nodes to repass (ordered):
  1.5
  2.1
  2.2
  4.1
  5.1
  6.2
Metrics:
  runtime_ms: 0.0318
  note: Baseline is a simple deterministic ordering (not a learning algorithm).
  ndcg_graded: 1.0000
  footrule_norm_vs_self: 0.0000
  inversion_rate_vs_self: 0.0000
  top3_overlap_vs_self: 1.0000
  top5_overlap_vs_self: 1.0000

— PageRank —
Nodes to repass (ordered):
  6.2
  1.5
  5.1
  2.1
  2.2
  4.1
  PageRank scores (candidates):
    6.2: 0.0304
    1.5: 0.0149
    5.1: 0.0148
    2.1: 0.0146
    2.2: 0.0146
    4.1: 0.0146
Metrics:
  runtime_ms: 2.4093
  spearman_vs_baseline: -0.2000
  kendall_tau_vs_baseline: -0.0667
  footrule_norm_vs_baseline: 0.7778
  inversion_rate_vs_baseline: 0.5333
  top3_overlap_vs_baseline: 0.3333
  top5_overlap_vs_baseline: 0.8000
  ndcg_graded_vs_baseline: 0.6755

— HITS (Authority) —
Nodes to repass (ordered):
  5.1
  6.2
  1.5
  2.1


<h1>Les commentaires</h1>

# === 1. Chargement du graphe et identification des nœuds à repasser ===
student_id = input("Entrez l'ID de l'étudiant : ")
# Ce bloc demande l'ID de l'étudiant (ex: 003) pour cibler un étudiant précis.
# Ensuite, le programme cherche les nœuds (modules) que l’étudiant doit repasser,
# c’est-à-dire les prérequis immédiats des modules échoués.

# === 2. Baseline (ordre déterministe) ===
baseline_nodes = sort_by_depth_and_digit_sum(candidates)
# Ici, on applique la baseline : ce n'est pas un algorithme d'apprentissage,
# juste un tri fixe selon deux critères : profondeur du nœud (depth ↑)
# puis somme des chiffres du nœud (digit_sum ↑).

print("Nodes to repass (ordered):", baseline_nodes)
# Résultat observé :
#  1.5 → 2.1 → 2.2 → 4.1 → 5.1 → 6.2
# Les métriques internes (ndcg_graded, footrule_norm_vs_self, etc.)
# sont toutes parfaites (1.0 ou 0.0) car on compare la baseline avec elle-même.
# Temps d’exécution : ~0.03 ms (quasi instantané, simple tri).

# === 3. PageRank ===
pagerank_scores = nx.pagerank(G)
pagerank_nodes = rank_candidates_by_score(candidates, pagerank_scores)
# PageRank calcule l’importance d’un nœud selon :
# - le nombre de liens entrants
# - et l’importance des nœuds qui pointent vers lui
# Plus un nœud reçoit des liens de nœuds eux-mêmes importants,
# plus son score est élevé.

print("Nodes to repass (ordered):", pagerank_nodes)
# Résultat observé :
#  6.2 → 1.5 → 5.1 → 2.1 → 2.2 → 4.1
# Le nœud 6.2 est le plus critique (score 0.0304).
# Corrélation Spearman avec la baseline : -0.200 (classement assez inversé).
# nDCG : 0.6755 → assez différent de la baseline mais conserve de bons candidats.
# Overlap Top3 : 0.3333 → un seul nœud du Top3 baseline est commun.
# Temps d’exécution : ~2.4 ms (plus long car calcul itératif).

# === 4. HITS (Authority) ===
hits_hubs, hits_authorities = nx.hits(G)
hits_nodes = rank_candidates_by_score(candidates, hits_authorities)
# HITS calcule deux scores par nœud :
#  - Authority : importance en tant que "référence" (cité par beaucoup d’autres).
#  - Hub : capacité à pointer vers d’autres nœuds importants.
# Ici on se base sur l’autorité.

print("Nodes to repass (ordered):", hits_nodes)
# Résultat observé :
#  5.1 → 6.2 → 1.5 → 2.1 → 2.2 → 4.1
# Le nœud 5.1 a l’autorité la plus forte (0.0970).
# Spearman vs baseline : -0.3714 (ordre encore plus inversé que PageRank).
# nDCG : 0.6003 → qualité plus faible que PageRank.
# Overlap Top5 : 0.8000 → presque tous les nœuds restent présents dans le Top5.
# Temps d’exécution : ~2.35 ms (proche de PageRank).

# === 5. Comparaison entre algorithmes ===
# On calcule la similarité des classements avec plusieurs métriques :
#  - Spearman : corrélation de rangs (1 = même ordre, -1 = ordre inverse).
#  - Kendall Tau : taux d’inversions de paires.
#  - Footrule : distance normalisée entre permutations.
#  - Top3/Top5 overlap : proportion d’éléments communs aux meilleurs rangs.

print("Baseline vs PageRank:", compute_spearman(baseline_nodes, pagerank_nodes))
print("Baseline vs HITS:", compute_spearman(baseline_nodes, hits_nodes))
print("PageRank vs HITS:", compute_spearman(pagerank_nodes, hits_nodes))
# Résultats observés :
#  - Baseline vs PageRank : Spearman = -0.200 (divergence modérée)
#  - Baseline vs HITS : Spearman = -0.371 (divergence plus forte)
#  - PageRank vs HITS : Spearman = 0.829 (forte similarité entre eux)

# === 6. Recommandation finale ===
best_algo = choose_best_algorithm([pagerank_metrics, hits_metrics])
# La baseline est volontairement exclue car ce n’est pas un algorithme d’apprentissage.
# Le choix se fait d’abord sur le nDCG, puis Spearman, puis la vitesse.
# PageRank est recommandé :
# - nDCG = 0.6755 (le plus élevé des vrais algorithmes)
# - Accord fort avec HITS (Spearman 0.829)
# - Temps d’exécution compétitif.
print(">> Recommendation:", best_algo)